In [182]:
# import necessary libraries
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math

In [183]:
# helpful to visualize the graph
def display_graph(graph_edges):
    
    # translate to networkx graph
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    # display the graph
    nx.draw(G, with_labels=True, font_weight='bold')
    plt.title('Undirected Unweighted Graph')
    plt.show()

Part 2 #4

In [184]:
# just gets the length of the array vertices
def get_number_of_vertices(graph_edges):
    return len(set(np.array(graph_edges).flatten()))

Part 2 #5

In [185]:
# helper function to remove duplicate edges
def get_unique_graph(graph_edges):
    for i in range(len(graph_edges)):
        edge = graph_edges[i]
        
        # sort edges to make sure they are in the same order
        if edge[0] > edge[1]:
            graph_edges[i] = (edge[1], edge[0])
    
    # remove duplicates
    unique_edges = set(graph_edges)
    
    return unique_edges

# counts connections from the target vertex
def get_vertex_degree(graph_edges, vertex):
    unique_edges = get_unique_graph(graph_edges)
    
    vertex_degree = 0
    # count the number of times the vertex appears in the unique edges
    for edge in unique_edges:
        if vertex in edge:
            vertex_degree += 1
    
    return vertex_degree
    

Part 2 #6

In [186]:
# helper function to calculate coefficient given the number of neighbors and actual connections
def calculate_clustering_coefficient(number_of_neighbors, actual_connections):
    if number_of_neighbors == 1:
        return 0
    
    # calculate the number of possible connections by using the combination formula
    possible_neighbors = math.comb(number_of_neighbors, 2)
    return actual_connections / possible_neighbors

# helper function to get all connections to the target vertex
def get_connections(graph_edges, vertex):
    edges = []
    
    # get all the edges that contain the target vertex
    for edge in graph_edges:
        if vertex in edge:
            edges.append(edge)
            
    return set(np.array(edges).flatten())

# helper function to get the subgraph of the target vertex
def get_subgraph(graph_edges, vertex):
    unique_edges = get_unique_graph(graph_edges)
    
    connections = get_connections(unique_edges, vertex)
        
    subgraph = []
    
    # get all the edges that contain the connections
    for edge in unique_edges:
        if all(num in connections for num in edge):
            subgraph.append(edge)
            
    return subgraph

# uses subgraph to get the number of connections to the vertex and calculate the coefficient
def get_vertex_clustering_coefficient(graph_edges, vertex):
    vertex_degree = get_vertex_degree(graph_edges, vertex)
    
    subgraph = get_subgraph(graph_edges, vertex)
    
    # count the number of actual connections
    actual_connections = len([pair for pair in subgraph if vertex not in pair])
    
    return calculate_clustering_coefficient(vertex_degree, actual_connections)
    

Part 2 #7

In [187]:
# method to get the betweenness centrality of a vertex by counting number of shortest paths
# note that this method returns normalized results
# only efficient on small graphs
def get_betweenness_centrality(graph_edges, vertex):
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    # get all the shortest paths in the graph
    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    target_betweenness_centrality = 0

    # sum the number of times the target vertex appears in the shortest paths
    for source, paths in shortest_paths.items():
        for end, path in paths.items():
            # only count if the vertex is not the source or end
            if vertex in path[1:-1]:
                # count the number of times the vertex appears in the path
                total_shortest_paths = len(shortest_paths[source][end])
                # add the count to the total
                target_betweenness_centrality += path.count(vertex) / total_shortest_paths
                
    # normalize result
    n = len(G.nodes())
    normalization_factor = ((n-1)*(n-2))/2
    target_betweenness_centrality /= normalization_factor

    return target_betweenness_centrality

# unnormalized version of the above method
def get_betweenness_centrality_unnormalized(graph_edges, vertex):
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    shortest_paths = dict(nx.all_pairs_shortest_path(G))

    target_betweenness_centrality = 0

    for source, paths in shortest_paths.items():
        for end, path in paths.items():
            if vertex in path[1:-1]:
                total_shortest_paths = len(shortest_paths[source][end])
                target_betweenness_centrality += path.count(vertex) / total_shortest_paths
                
    return target_betweenness_centrality

Part 2 #8

In [188]:
# method to get the average shortest path length of the graph
def get_average_shortest_path_length(graph_edges):
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    
    shortest_path_lengths = []
    nodes = G.nodes()
    
    # get the shortest path length between all pairs of nodes
    for node1 in nodes:
        for node2 in nodes:
            if node1 != node2:
                shortest_path_lengths.append(nx.shortest_path_length(G, source=node1, target=node2))

    # return the average of all the shortest path lengths
    average_shortest_path_length = sum(shortest_path_lengths) / len(shortest_path_lengths)
    return average_shortest_path_length
    
    

Part 2 #9

In [189]:
# NOTE this function assumes a 0-indexed graph and treats all as such

# method to get the full adjacency matrix of the graph
def get_adjacency_matrix(graph_edges):
    num_nodes = max(max(edge) for edge in graph_edges) + 1
    
    # initialize the adjacency matrix
    adjacency_matrix = np.zeros((num_nodes, num_nodes))
    
    # fill in the adjacency matrix
    for edge in graph_edges:
        node1, node2 = edge
        adjacency_matrix[node1][node2] = 1
        adjacency_matrix[node2][node1] = 1  
    
    return adjacency_matrix

Part 2 #10

In [190]:
# power iteration to find the eigenvector centrality of each node in a network
def get_eigenvector_centrality(adjacency_matrix, iterations=100):
    # initialize the eigenvector
    eigenvector = np.ones(len(adjacency_matrix))
    eigenvector = eigenvector / np.linalg.norm(eigenvector)
    
    # power iteration
    for i in range(iterations):
        # multiply the adjacency matrix by the eigenvector
        eigenvector = np.dot(adjacency_matrix, eigenvector)
        # normalize the eigenvector
        eigenvector = eigenvector / np.linalg.norm(eigenvector)
    
    return eigenvector


In [191]:
# main function to test the above methods
def main():
    # example input graph
    input_graph = [
        (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1,10),
        (2, 7), (2,8), (2,9),
        (3, 4), (3, 6), (3, 8),
        (4, 8),
        (5, 7),
        (6, 8),
        (7, 8), (7, 9),
        (8, 9), (8, 10),
        (9, 10),
        (10, 11),
        (2, 1),
        (10, 11),
        (7, 1)
    ] 
    
    # call the functions and print the results for each one
    print("Number of vertices:", get_number_of_vertices(input_graph)) # 11
    print("Vertex degree:", get_vertex_degree(input_graph, 1)) # 6
    print("Vertex clustering coefficient:", get_vertex_clustering_coefficient(input_graph, 1)) # 0.190
    print("Betweenness centrality:", get_betweenness_centrality(input_graph, 1)) # ?
    print("Betweenness centrality (unnormalized):", get_betweenness_centrality_unnormalized(input_graph, 1)) # ?
    print("Average shortest path length:", get_average_shortest_path_length(input_graph)) # 1.709
    print("Adjacency matrix:")
    print(get_adjacency_matrix(input_graph))
    print("Eigenvector centrality:")
    print(get_eigenvector_centrality(get_adjacency_matrix(input_graph)))

    


main()

Number of vertices: 11
Vertex degree: 7
Vertex clustering coefficient: 0.19047619047619047
Betweenness centrality: 0.2833333333333334
Betweenness centrality (unnormalized): 12.750000000000002
Average shortest path length: 1.709090909090909
Adjacency matrix:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Eigenvector centrality:
[0.         0.41251858 0.32767958 0.29270246 0.2484588  0.16680067
 0.2484588  0.35738217 0.44158863 0.30097909 0.26257683 0.05688784]
